In [18]:
import h5py
import tabulate
import contextily as ctx
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import random
import math
from datetime import datetime, timedelta
# from IPython.display import HTML, display
from os import path
from shapely.geometry import Point
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.lines import Line2D
from matplotlib import cm
from matplotlib.colors import ListedColormap
from scipy import stats
import time
import multiprocessing
from multiprocessing import Pool
from rasterio.plot import show
import functools
from osgeo import ogr
from osgeo import gdal
import os
cwd = os.getcwd()
print(cwd)
from glob import glob
from os import path

/gpfs/data1/duncansongp/amberliang/EA_data


In [19]:
topo = gdal.Open('/gpfs/data1/duncansongp/amberliang/EA_data/ancillary_vars/topo_stack.tif')
print(topo.GetRasterBand(3).GetDescription())

aspect


In [20]:

def get_raster_value(geo_x, geo_y, ds, band_index):
#     """Return raster value that corresponds to given coordinates."""
    forward_transform = ds.GetGeoTransform()
    pixel_width = forward_transform[1]
    pixel_height = -forward_transform[5]    
    reverse_transform = gdal.InvGeoTransform(forward_transform)
    pixel_coord = gdal.ApplyGeoTransform(reverse_transform, geo_x, geo_y)
    pixel_x = math.floor(pixel_coord[0])
    pixel_y = math.floor(pixel_coord[1])
    band = ds.GetRasterBand(band_index)
    val_arr = band.ReadAsArray(pixel_x, pixel_y, 1, 1) # Avoid reading the whole raster into memory - read 1x1 array
    if val_arr is None:
#         pass
#         print(out)
        return np.nan
    else:
        out=val_arr[0][0]
#         print(out)
        return out
def bname(ds, band_index):
    return  ds.GetRasterBand(band_index).GetDescription()

def getEx(pt, ds):
    from shapely.geometry import Polygon
    """ Return list of corner coordinates from a gdal Dataset """
    xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
    width, height = ds.RasterXSize, ds.RasterYSize
    xmax = xmin + width * xpixel
    ymin = ymax + height * ypixel
    
    poly=Polygon(((xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)))
    return poly.contains(pt)  #return true or false 

def ls_Extract(tileX, yr, biome, cornerInd):
    #tileX=tileX[0]
    print('tile', tileX)
    print('yr', yr)
    print('biome', biome)
    yr=str(yr)
    biome=str(biome)

    imageStack = "ls_lhs_medoid_"+tileX+"_"+yr+"_NBR_3by3_dur8_biome" + biome+ "_val_v2-"+'*.tif'  ####!!![need to modify]!!!###
    
    out="/tilelevel_tile_"+tileX+"_cor"+str(cornerInd)+ '_biom'+biome+'_'+yr+'_topo_lhs_nbr_33_dur8.csv'  ####!!![need to modify]!!!###

    #preping the sample dataframe
    pdir= "/gpfs/data1/duncansongp/amberliang/EA_data/landsat_comp/t"+tileX+'/'+yr   #EA_data/gd_data/  t168050_1986-2021\2019
    # pdir= "/gpfs/data1/duncansongp/amberliang/gd2/all_metrics_NBR_fitted"
    outdir = '/gpfs/data1/duncansongp/amberliang/EA_data/ls_ltr_training/tile'+tileX+'_filtered_l4a_ls_output_corner'#biome'+str(biome)+'_'+yr
    annual_pl= glob.glob(path.join(pdir, imageStack))[cornerInd]  

    samples= pd.read_csv('samples/tigray_gedi_samples_biome1_'+yr+'_highval.csv')     ####!!![need to modify]!!!
#     samples =pd.read_csv("samples/"+"biome"+str(wwfbiome)+"_topo_training_l4a_l2b_lhs_nbr_33_dur8_empty_ndvi_"+yr+".csv")
    print('total samples', samples.shape)
    sample2020_min_1 = samples#.iloc[0:100]# [(samples['biome']==int(biome))]#cwd + '/' + "kibale_lite_for_gee_nicfi_"+yr+".csv") 
    print('sub samples', sample2020_min_1.shape)
    #     sample2020_min_1['shot_number']= sample2020_min_1['shot_number'].astype(str)
#     print(sample2020_min_1.dtypes)
    sample2020_min_1_gdf= gpd.GeoDataFrame(sample2020_min_1, 
                                           geometry=gpd.points_from_xy(sample2020_min_1.lon_lowestmode, sample2020_min_1.lat_lowestmode), 
                                           crs="EPSG:4326")
    #add in tile information to the sample gdf for easy filtering in later step
#     join = gpd.sjoin(sample2020_min_1_gdf, polys, how='left',op="within")
#     sample2020_min_1_gdf = join[join['Name'].notnull()]
#     sample2020_min_1_gdf=sample2020_min_1_gdf.drop(['descriptio', 'timestamp', 'begin', 'end', 'altitudeMo',
#            'tessellate', 'extrude', 'visibility', 'drawOrder', 'icon', 'geometry','index_right'], axis=1)
#     sample2020_min_1_gdf['Name']=sample2020_min_1_gdf['Name'].str.replace('_', '0')
#     biomeYrDF= gpd.GeoDataFrame(sample2020_min_1_gdf, geometry=gpd.points_from_xy(sample2020_min_1_gdf.lon_lowestmode, sample2020_min_1_gdf.lat_lowestmode), crs="EPSG:4326")

    sample2020_min_1_gdf_tile=sample2020_min_1_gdf
    print(sample2020_min_1_gdf_tile.shape)
#     sample2020_min_1_gdf_tile = biomeYrDF[biomeYrDF['Name'] == tileX]
#     year ="allVI_NBR_fit_"+yr+"_dur_10_PW_"+tileX+"_ALL"+'*.tif'
    
#     print("Tif files for each tile of each year", list(annual_pl))
    print('number of subset in tile: '+str(cornerInd))
    
    df = pd.DataFrame()
    for rcount in range(0,1):#range(0,len(annual_pl)):
        print('raster', rcount)
        src=gdal.Open(annual_pl)#gdal.Open(annual_pl[rcount])
        topo = gdal.Open('/gpfs/data1/duncansongp/amberliang/EA_data/ancillary_vars/topo_stack.tif')
        i = 0
        while i < len(sample2020_min_1_gdf_tile):
            
            geo_x, geo_y = sample2020_min_1_gdf_tile.iloc[i]['lon_lowestmode'],sample2020_min_1_gdf_tile.iloc[i]['lat_lowestmode'] #coord_list[i][0], coord_list[i][1]
            geom =  sample2020_min_1_gdf_tile.iloc[i].geometry  #convert to point with geom for checking if its in the ras ex
            outlist = {}
            outlist['shot_number']=  sample2020_min_1_gdf_tile.iloc[i]['shot_number'].astype(str)
            outlist['agbd']=  sample2020_min_1_gdf_tile.iloc[i]['agbd']
            outlist['lon_lowestmode']=  sample2020_min_1_gdf_tile.iloc[i]['lon_lowestmode']
            outlist['lat_lowestmode']=  sample2020_min_1_gdf_tile.iloc[i]['lat_lowestmode']
            if getEx(geom, src):
#                 print('point in this subset')
                print(i)
                for b in range(0, src.RasterCount):
                    bind=b+1
                    out0=get_raster_value(geo_x, geo_y, ds=src, band_index=bind)
#                     out00= get_raster_value(geo_x, geo_y, ds=topo, band_index=bind)
#                     print("extracted",out0)
    #                 print(bname(src, bind))
                    if out0 is None:
                        out0=np.nan
    #                 print(out0)

                    outlist[bname(src, bind)]=out0
                for t in range(0, topo.RasterCount):
                    tind=t+1
                    
                    out00= get_raster_value(geo_x, geo_y, ds=topo, band_index=tind)
#                     print("extracted",out0)
    #                 print(bname(src, bind))
                    if out00 is None:
                        out0=np.nan
    #                 print(out0)

                    outlist[bname(topo, tind)]=out00
#                 print(outlist)
                if not (all(np.isnan(value) for value in list(outlist.values())[4:])):
#                     print(all(np.isnan(value) for value in list(outlist.values())[1:]))
                    df = df.append(outlist, ignore_index=True)
                i+=1
            else: 
                i+=1
    print(df.shape)
    print(df)
    
    if len(df)>0:
#         sample2020_min_1_gdf_tile_out= sample2020_min_1_gdf_tile.join(df, on="shot_number",how='left')
#         print(sample2020_min_1_gdf_tile_out.shape)
        df = df[(df['ftv_ndvi_fit']!=0) & (df['GDMAG']!=0)]
        print(str(len(df))+" out of total "+ str(len(sample2020_min_1_gdf_tile))+" were extracted")
        df.to_csv(outdir+out, index=False)
        return(df)
#         return df
    else:
        return df


In [23]:
# shp="AOI/EA_wgs_tiles.shp"
# polys = gpd.read_file(shp)
import glob
wwfbiome=1
biomeTiles= ['168060']#['173061','173060', '173059', '172061','172060', '172059']#list(pd.read_csv("TSMBF_WGS2_tiles.csv")['Name'].str.replace('_', '0'))# ['169055']
yrs= [ '2021','2020','2019']

corners=range(0,16)
biome=str(wwfbiome)
# args = [(biomeTiles, yr, biome) for biomeTiles in biomeTiles]
args = [(biomeTiles, yr, biome, cornerInd) for biomeTiles in biomeTiles for yr in yrs for cornerInd in corners]
print(len(args))
print(args)

ls_Extract('168060', '2020', '1', 1)

# parallel processing 
# def main():
#     tic = time.time()
#     pool = Pool(processes=12)  # set the processes max number 3
#     result = pool.starmap(ls_Extract, args)
#     pool.terminate()
#     pool.join()
#     print(result)
#     print('end')
#     toc = time.time()
#     print('Done in {:.4f} seconds'.format(toc-tic))

    
# if __name__ == "__main__":
#     main()

48
[('168060', '2021', '1', 0), ('168060', '2021', '1', 1), ('168060', '2021', '1', 2), ('168060', '2021', '1', 3), ('168060', '2021', '1', 4), ('168060', '2021', '1', 5), ('168060', '2021', '1', 6), ('168060', '2021', '1', 7), ('168060', '2021', '1', 8), ('168060', '2021', '1', 9), ('168060', '2021', '1', 10), ('168060', '2021', '1', 11), ('168060', '2021', '1', 12), ('168060', '2021', '1', 13), ('168060', '2021', '1', 14), ('168060', '2021', '1', 15), ('168060', '2020', '1', 0), ('168060', '2020', '1', 1), ('168060', '2020', '1', 2), ('168060', '2020', '1', 3), ('168060', '2020', '1', 4), ('168060', '2020', '1', 5), ('168060', '2020', '1', 6), ('168060', '2020', '1', 7), ('168060', '2020', '1', 8), ('168060', '2020', '1', 9), ('168060', '2020', '1', 10), ('168060', '2020', '1', 11), ('168060', '2020', '1', 12), ('168060', '2020', '1', 13), ('168060', '2020', '1', 14), ('168060', '2020', '1', 15), ('168060', '2019', '1', 0), ('168060', '2019', '1', 1), ('168060', '2019', '1', 2), ('16

IndexError: list index out of range

In [10]:
args[1]

('169051', '2020', '1', 1)

In [24]:
tileX= '168060'
yr ='2021'
biome='1'
cornerInd =1
print('tile', tileX)
print('yr', yr)
print('biome', biome)
yr=str(yr)
biome=str(biome)

imageStack = "ls_lhs_medoid_"+tileX+"_"+yr+"_NBR_3by3_dur8_biome" + biome+ "_val_v2-"+'*.tif'  ####!!![need to modify]!!!###

out="/tilelevel_tile_"+tileX+"_cor"+str(cornerInd)+ '_biom'+biome+'_'+yr+'_topo_lhs_nbr_33_dur8.csv'  ####!!![need to modify]!!!###

#preping the sample dataframe
pdir= "/gpfs/data1/duncansongp/amberliang/EA_data/landsat_comp/t"+tileX+'/'+yr   #EA_data/gd_data/  t168050_1986-2021\2019
# pdir= "/gpfs/data1/duncansongp/amberliang/gd2/all_metrics_NBR_fitted"
outdir = '/gpfs/data1/duncansongp/amberliang/EA_data/ls_ltr_training/tile'+tileX+'_filtered_l4a_ls_output_corner'#biome'+str(biome)+'_'+yr
annual_pl= glob.glob(path.join(pdir, imageStack))[cornerInd] 


tile 168060
yr 2021
biome 1


IndexError: list index out of range

In [27]:
path.join(pdir, imageStack)

'/gpfs/data1/duncansongp/amberliang/EA_data/landsat_comp/t168060/2021/ls_lhs_medoid_168060_2021_NBR_3by3_dur8_biome1_val_v2-*.tif'

In [25]:
glob.glob(path.join(pdir, imageStack))

[]